In [1]:
print("DLModel")


import pandas as pd
from numpy import mean
from numpy import std
from numpy import asarray
from sklearn.datasets import make_regression
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense

max_models = 2
res_lst = []
model_lst = []


DLModel


In [2]:
pd.set_option('display.max_columns', 200)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:


# get the dataset
def load_data(how_many=0):
	if how_many == 0:
		samples = 100
	elif how_many == 1:
		samples = 1000
	elif how_many == 2:
		samples = 5000

	X, y = make_regression(n_samples=samples, n_features=10, n_informative=5, n_targets=3, random_state=42)
	print("samples: ", samples)
	return X, y

In [4]:
def load_random_data():
    df = pd.read_csv('../data/random_data.csv')
    df.describe()
    X = df[["Var1", "Var2", "Var3", "Var4", "Var5", "Var6"]]
    y = df[["Var7", "Var8", "Var9"]]
    return X, y

In [5]:
X, y = load_random_data()
print(X)


            Var1       Var2       Var3       Var4       Var5       Var6
0      44.983971  33.933642  27.750606  16.755403  12.846325  18.698162
1      82.072771  53.940198  18.263558  12.617179  60.213736  36.162777
2      73.802128  98.593598  11.908945  63.936180  40.649786  92.163184
3      49.079078  66.976854  81.216915  86.776962  79.616895  87.328358
4      82.973305  30.057045  26.142513  37.337483  85.298177  44.467624
...          ...        ...        ...        ...        ...        ...
49995  96.576127  83.218668  74.394776  91.983234  27.793881  45.727292
49996  76.274775  84.413582  65.935641  42.844653  87.987294  45.051603
49997  63.870936  94.921410  48.270378  10.509027  86.829932  65.622355
49998  61.200710  69.917416  97.190127  22.265976  20.210938  12.279141
49999  15.445123  16.654922  28.629873  61.661492  81.698829  65.645658

[50000 rows x 6 columns]


In [6]:


# get the model
def get_model(n_inputs, n_outputs, num):
	if num == 0:
		lcl_model = Sequential()
		lcl_model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
		lcl_model.add(Dense(n_outputs))
		lcl_model.compile(loss='mae', optimizer='adam')
	elif num == 1:
		lcl_model = Sequential()
		lcl_model.add(Dense(22, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
		lcl_model.add(Dense(n_outputs))
		lcl_model.compile(loss='mae', optimizer='adam')
	return lcl_model

In [7]:
# evaluate a model using repeated k-fold cross-validation
def evaluate_model(X, y, model_num):
	results = list()
	models = []

	# define evaluation procedure
	cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
	# enumerate folds
	for train_ix, test_ix in cv.split(X):
		# prepare data
		n_inputs, n_outputs = X.shape[1], y.shape[1]
		X_train, X_test = X[train_ix], X[test_ix]
		y_train, y_test = y[train_ix], y[test_ix]

		# define model
		model = get_model(n_inputs, n_outputs, model_num)
		# fit model
		model.fit(X_train, y_train, verbose=0, epochs=5)
		# evaluate model on test set
		mae = model.evaluate(X_test, y_test, verbose=0)
		# store result
		# print('model_num: %d metrics_names: %s mae>%.3f' % (model_num, model.metrics_names, mae))
		results.append(mae)
		models.append(model)
	return mean(results), models[0]





In [8]:
# For each test case
for test_case in range(3):
	# load dataset
	X, y = load_data(test_case)

	# Let's try each model
	best_result = 0.0
	best_model = None
	for curr_model in range(max_models):
		# evaluate model
		result, model = evaluate_model(X, y, curr_model)
		res_lst.append(result)
		model_lst.append(model)
		print('Model Number: %d metrics_names: %s Result: %.3f Model address: %s' % (curr_model, model.metrics_names, result, model))
		if result > best_result:
			best_result = result
			best_model = model


	print("best_result: %.3f best_model %s" % (best_result, best_model))

samples:  1000
Model Number: 0 metrics_names: ['loss'] Result: 82.210 Model address: <keras.engine.sequential.Sequential object at 0x00000261399399A0>
Model Number: 1 metrics_names: ['loss'] Result: 82.115 Model address: <keras.engine.sequential.Sequential object at 0x0000026145FCDD60>
best_result: 82.210 best_model <keras.engine.sequential.Sequential object at 0x00000261399399A0>
samples:  5000
Model Number: 0 metrics_names: ['loss'] Result: 92.642 Model address: <keras.engine.sequential.Sequential object at 0x000002615B201D90>
Model Number: 1 metrics_names: ['loss'] Result: 91.241 Model address: <keras.engine.sequential.Sequential object at 0x0000026155A67640>
best_result: 92.642 best_model <keras.engine.sequential.Sequential object at 0x000002615B201D90>
samples:  10000
Model Number: 0 metrics_names: ['loss'] Result: 49.517 Model address: <keras.engine.sequential.Sequential object at 0x0000026157D616A0>
Model Number: 1 metrics_names: ['loss'] Result: 46.372 Model address: <keras.eng

In [10]:
# make a prediction for new data
row = [-0.99859353,2.19284309,-0.42632569,-0.21043258,-1.13655612,-0.55671602,-0.63169045,-0.87625098,-0.99445578,-0.3677487]
newX = asarray([row])
yhat = best_model.predict(newX)
print('Predicted: %s' % yhat[0])

1/1 [==============================] - 0s 42ms/step
Predicted: [-17.384346 -50.32769   24.73497 ]
